In [ ]:
# dependencies import
from common_dependencies import *
from tensorboard.plugins.hparams import api as hp
logger = logging.getLogger(f'main.ae_model_train')

# Чтение и подготовка данных

### Параметры обучающих выборок

In [ ]:
# train params that can be easily changed
MAX_VAL = 1000 # не трогать
XSHIFT = 200 # не трогать

# настройка параметров выборок
dataset_desc = {'train': (DatasetPartDescription(PATH_TO_DATA['run_1'],DataCrop(0,0,MAX_VAL,60),SlidingCrop(1,1),XSHIFT),
                          DatasetPartDescription(PATH_TO_DATA['run_2'],DataCrop(0,0,MAX_VAL,60),SlidingCrop(1,1),XSHIFT),
                         ),
                'val': (DatasetPartDescription(PATH_TO_DATA['run_1'],DataCrop(0,60,MAX_VAL,20),SlidingCrop(1,1),XSHIFT),
                        DatasetPartDescription(PATH_TO_DATA['run_2'],DataCrop(0,60,MAX_VAL,20),SlidingCrop(1,1),XSHIFT)
                       ),
                'test': (DatasetPartDescription(PATH_TO_DATA['run_1'],DataCrop(0,80,MAX_VAL,MAX_VAL),SlidingCrop(1,1),XSHIFT),
                         DatasetPartDescription(PATH_TO_DATA['run_2'],DataCrop(0,80,MAX_VAL,MAX_VAL),SlidingCrop(1,1),XSHIFT)
                        )}

### Чтение и подготовка данных обучающих выборок

In [ ]:
#входные и выходные данные
# reading
dataset = {'train':dict(zip(['x','y','bin'], [np.array(list(gen)) for gen in chain_dataset_gens(dataset_desc['train'])])),
           'val':dict(zip(['x','y','bin'], [np.array(list(gen)) for gen in chain_dataset_gens(dataset_desc['val'])])), 
           'test':dict(zip(['x','y','bin'], [np.array(list(gen)) for gen in chain_dataset_gens(dataset_desc['test'])]))}

# squueze datasets
for dataset_part_name, dataset_part in dataset.items():
    for data_part_name, data_part in dataset_part.items():
        if data_part_name == 'x':
            dataset[dataset_part_name][data_part_name] = data_part.reshape(-1,64)
        if data_part_name == 'y':
            dataset[dataset_part_name][data_part_name] = data_part.reshape(-1)

### Размерности данных в обучающих выборках

In [ ]:
# displaying
logger.debug('Dataset')
for dataset_part_name, dataset_part in dataset.items():
    logger.debug('|'*8+dataset_part_name+'|'*8)
    for data_part_name, data_part in dataset_part.items():
        logger.debug(f'{data_part_name}.shape: {data_part.shape}')

### Какие части 2 файлов с данными относятся к конкретным выборкам (к тренировочной, тестовой, валидационной)

In [ ]:
# show parts took for learning
all_rects = {'run_1': {'train':None,'val':None,'test':None}, 
             'run_2': {'train':None,'val':None,'test':None}}
rects_colors = {'train':'red', 'val':'green', 'test':'yellow'}

for run_name in all_rects.keys():
    x_df, y_df = dw.get_x_and_y_data(*PATH_TO_DATA[run_name])
    x_df = None
    y_df = dw.roll_df(y_df, XSHIFT, 1)
    for dataset_part_name in all_rects[run_name].keys():
        # get all DatasetPartDescription for train, val or test
        dataset_part_desc = dataset_desc[dataset_part_name]
        # get all DatasetPartDescription for current run_name (run_1 or run_2)
        dataset_part_desc = [dataset_part for dataset_part in dataset_part_desc if re.findall(r'run_\d', dataset_part.data_path_tuple[0])[0] == run_name]
        # put rects list to all_rects[run_name][dataset_part_name]
        all_rects[run_name][dataset_part_name] = [Rectangle((dataset_part.file_data_crop.left, dataset_part.file_data_crop.top), 
                           dataset_part.file_data_crop.width, dataset_part.file_data_crop.height, 
                           facecolor=rects_colors[dataset_part_name], alpha=0.5) for dataset_part in dataset_part_desc]
    res_rects = list(itertools.chain(*[run_rects for run_rects_name, run_rects in all_rects[run_name].items()]))
    if res_rects:
        dw.draw_defects_map_with_rectangles_owerlap(y_df, res_rects, title = f'The parts took for learning from {run_name} (red - train, green - validate, yellow - test)')

# Создание и обучение модели автокодировщика

In [ ]:
# архитектура модели
def get_model(learning_rate):
    #CMP_learning_rate = 0.005 #0.0000002 # шаг сходимости back propogation
    CMP_solver = keras.optimizers.Adam(learning_rate) # оптимизатор
    CMP_loss_funcs = keras.losses.MeanSquaredError() #BinaryCrossentropy() 
    CMP_metrics = [keras.metrics.MeanSquaredError(name='MeanSquaredError')]
    
    HIDDEN_ACTIVATION = 'sigmoid' #'relu'
    OUTPUT_ACTIVATION = 'sigmoid'
    
    enc_input = layers.Input((64,), name='enc_input')
    d_1_1 = layers.Dense(64, activation=HIDDEN_ACTIVATION)(enc_input)
    
    d_2_1 = layers.Dense(32, activation=HIDDEN_ACTIVATION)(d_1_1)
    d_2_2 = layers.Dense(32, activation=HIDDEN_ACTIVATION)(d_1_1)
    #d_2_3 = layers.Dense(8, activation=HIDDEN_ACTIVATION)(d_1_1)

    #d_4_1 = layers.Dense(16, activation=HIDDEN_ACTIVATION)(concatenate([d_2_1, d_2_2], axis=1))
    #d_4_3 = layers.Dense(32, activation=HIDDEN_ACTIVATION)(d_1_1)

    hidden_state_output = layers.Dense(8, activation=HIDDEN_ACTIVATION, name='hidden_state_output')(concatenate([d_2_1, d_2_2], axis=1))

    #d_5_1 = layers.Dense(16, activation=HIDDEN_ACTIVATION)(hidden_state_output)
    

    d_7_1 = layers.Dense(32, activation=HIDDEN_ACTIVATION)(hidden_state_output)
    d_7_2 = layers.Dense(32, activation=HIDDEN_ACTIVATION)(hidden_state_output)
    #d_7_3 = layers.Dense(8, activation=HIDDEN_ACTIVATION)(d_6_2)


    dec_output = layers.Dense(64, activation=OUTPUT_ACTIVATION, name='dec_output')(concatenate([d_7_1,d_7_2], axis=1))
    
    model = keras.Model(enc_input, dec_output, name='ae')
    model.compile(optimizer=CMP_solver, loss=CMP_loss_funcs, metrics=CMP_metrics)
    
    return model

### Вывести параметры модели и её граф

In [ ]:
model = get_model(0.005)
print(model.summary())
'''tf.keras.utils.plot_model(
    model,
    show_shapes=True,
    show_dtype=False,
    show_layer_names=True,
    rankdir="TB",
    expand_nested=False,
    dpi=200,
    show_layer_activations=False,
    show_trainable=False,
)''';

### Обучение модели

In [ ]:
# до какого размера кодируются данные
# или по имени слоя. Обычно "hidden_state_output"
# model.get_layer('hidden_state_output').output.shape[1]
ENCODED_SIZE = min([layer.output.shape[1] for layer in model.layers]) 
# размер входных и выходных данных
DECODED_SIZE = model.layers[-1].output.shape[1]
PATH_TO_SAVE_MODEL = pathlib.Path(f'networks/AE/encoded_to_{ENCODED_SIZE}')
PATH_TO_SAVE_MODEL_PROGRESS = PATH_TO_SAVE_MODEL/'logs'
MODEL_VERSION = 1
MODEL_NUMBER = 1
MIN_TRAIN_LOSS = 1
MIN_VAL_LOSS = 1

if not os.path.exists(PATH_TO_SAVE_MODEL):
    os.makedirs(PATH_TO_SAVE_MODEL)

if not os.path.exists(PATH_TO_SAVE_MODEL_PROGRESS):
    os.makedirs(PATH_TO_SAVE_MODEL_PROGRESS)

# все имеющиеся модели с такими же ENCODED_SIZE и DECODED_SIZE
all_ae_models = [path.name for path in PATH_TO_SAVE_MODEL.parent.rglob('*.keras') 
                 if re.search(fr'in\({DECODED_SIZE}\)_hid\({ENCODED_SIZE}\)', path.name)]

# если уже есть такая же архитектура модели сделать MODEL_VERSION такой же
# а MODEL_NUMBER на 1 больше чем имеющаяся
if all_ae_models:
    min_train_loss = min([float(re.findall(fr'train=(.+),val', name)[0]) for name in all_ae_models])
    min_val_loss = min([float(re.findall(fr',val=(.+),test', name)[0]) for name in all_ae_models])
    min_model_number = max([int(re.findall(fr'id=v{MODEL_VERSION:04}n(\d+)_', name)[0]) for name in all_ae_models])

    if MODEL_NUMBER <= min_model_number:
        MODEL_NUMBER = min_model_number+1
        
    if MIN_TRAIN_LOSS >= min_train_loss:
        MIN_TRAIN_LOSS = min_train_loss

    if MIN_VAL_LOSS >= min_val_loss:
        MIN_VAL_LOSS = min_val_loss

print(f'{ENCODED_SIZE=}')
print(f'{DECODED_SIZE=}')
print(f'{PATH_TO_SAVE_MODEL=}')
print(f'{MODEL_VERSION=}')
print(f'{MODEL_NUMBER=}')
print(f'{MIN_TRAIN_LOSS=}')
print(f'{MIN_VAL_LOSS=}')

### Настройка и создание коллбэков

In [ ]:
callback_params = {
    # остановка обучения если модель перестала учиться
    'EarlyStopping': {
        'monitor': 'val_loss', # отслеживаемый параметр 
        'min_delta': 0.00001, # минимальное улучшение параметра за cur_patience
        'patience': 6, # кол-во эпох без улучшений
        'restore_best_weights': False,  # сохранять ли веса нейронки с лучшими результатами
    },

    # уменьшение шага сходимости, если модель стала мендленно учиться
    'ReduceLROnPlateau': {
        'monitor' : 'loss', # отслеживаемый параметр 
        'factor' : 0.2, # множитель для расчета нового шага сходимости (new_learning_rate = old_learning_rate*RLPOP_factor)
        'patience' : 3, # кол-во эпох без улучшений
        'verbose' : 0, # выводить ли прогресс изменения шага сходимости в его процессее
        'min_delta' : 0.0001, # порог изменения отслеживаемого значения
        'cooldown' : 1, # количество эпох до возобновления работы после изменения шага сходимости
        'min_lr' : 0# минимальное значение шага сходимости
    },
}

# запись процесса обучения в логи
''''TensorBoard': {
    'log_dir' : PATH_TO_SAVE_MODEL/'logs',
    'histogram_freq' : 1,
    'write_images' : 1,
    'embeddings_freq' : 1
}''';

In [ ]:
# Создание и настройка колбэков

callback_list = [] # массив колбэков до подачи в колбек "callbacklist"

# остановка обучения если модель перестала учиться
callback_list.append(keras.callbacks.EarlyStopping(**callback_params['EarlyStopping']))

# уменьшение шага сходимости, если модель стала мендленно учиться
callback_list.append(keras.callbacks.ReduceLROnPlateau(**callback_params['ReduceLROnPlateau']))

# запись процесса обучения в логи
#callback_list.append(keras.callbacks.TensorBoard(**callback_params['TensorBoard']))

### Обучение моделей

In [ ]:
for seed in range(0, 200, 10):
    print(f'Seed: {seed}',"|"*10)
    tf.compat.v1.set_random_seed(seed)
    tf.random.set_seed(seed)
    np.random.seed(seed)
    for lrate in [0.01,0.005,0.0025]:
        print(f'Start learning rate: {lrate}',"|"*5)
        for batch_size in [8,16,32,64]:
            print(f'\tBatch size: {batch_size}')

            # get model
            model = get_model(lrate)
            
            history = model.fit(dataset['train']['x'], dataset['train']['x'],
                            batch_size = batch_size, 
                            epochs = 60, 
                            verbose = 0, 
                            shuffle = True,
                            validation_data = (dataset['val']['x'], dataset['val']['x']), 
                            callbacks = callback_list)
            cur_test_loss = model.evaluate(dataset['test']['x'], dataset['test']['x'], 
                                      batch_size=batch_size, verbose=0)[0]

            cur_train_loss = history.history['loss'][-1]
            cur_val_loss = history.history['val_loss'][-1]
            
            model_name = (
                f"id=v{MODEL_VERSION:04}n{MODEL_NUMBER:04}" +
                f"_in({DECODED_SIZE})_hid({ENCODED_SIZE})" + 
                f"_loss_MSE=(train={cur_train_loss:.5f}," + 
                f"val={cur_val_loss:.5f},test={cur_test_loss:.5f})" + 
                f"_seed={seed}_lrate={lrate}_bach_size={batch_size}")


            if cur_train_loss < MIN_TRAIN_LOSS and cur_val_loss < MIN_VAL_LOSS:
                MIN_TRAIN_LOSS = cur_train_loss
                MIN_VAL_LOSS = cur_val_loss
                cont = True
            if cur_val_loss < MIN_VAL_LOSS:
                MIN_VAL_LOSS = cur_val_loss
                cont = True
            if cur_train_loss < MIN_TRAIN_LOSS:
                MIN_TRAIN_LOSS = cur_train_loss
                cont = True
                
            print(f'\t\tEpochs: {len(history.history["loss"])}')
            print(f"\t\tloss_MSE=(train={cur_train_loss:.5f},val={cur_val_loss:.5f},test={cur_test_loss:.5f})")
            
            if cont:
                learn_df = pd.DataFrame.from_dict(history.history)
                callback_df = pd.DataFrame.from_dict(callback_params)
                dataset_desc_df = pd.DataFrame.from_dict(dataset_desc)
                dataset_df = pd.DataFrame.from_dict({part_name: {item_name: item.shape for (item_name, item) in part.items()} 
                                                         for (part_name, part) in dataset.items()})
                with pd.ExcelWriter(PATH_TO_SAVE_MODEL_PROGRESS/f"{model_name}_learning_data.xlsx") as writer:
                    learn_df.to_excel(writer, sheet_name = 'learning_progress')
                    callback_df.to_excel(writer, sheet_name = 'callback_params')
                    dataset_desc_df.to_excel(writer, sheet_name = 'dataset_reading_params')
                    dataset_df.to_excel(writer, sheet_name = 'prepared_dataset_params')
                    
                model.save(PATH_TO_SAVE_MODEL/f'{model_name}.keras')
                
                MODEL_NUMBER+=1
                cont=False
    print()

In [ ]:
'''#id=v13n01_in(64)_hid(16)_loss_MSE=(train=0.00077,val=0.00054,test=0.00075)
MODEL_VER = 15
min_train_loss = 0.00141
min_val_loss = 0.00098
num=1
for i in range(0,1000,10):
    tf.compat.v1.set_random_seed(i)
    tf.random.set_seed(i)
    np.random.seed(i)
    print(f'Seed: {i}',"|"*10)
    for learning_rate in [0.01,0.005]:
        print(f'Start learning rate: {learning_rate}',"|"*5)
        for batch_size in [32,]:
            print(f'\tBatch size: {batch_size}')


            model = get_model()
            history = model.fit(dataset['train']['x'], dataset['train']['x'],
                            batch_size = batch_size, 
                            epochs = 60, 
                            verbose = 0, 
                            shuffle=True,
                            validation_data = (dataset['val']['x'], dataset['val']['x']), 
                            callbacks = callback_list)
            res_loss = model.evaluate(dataset['test']['x'], dataset['test']['x'], batch_size = 32, verbose=0)
            
            cont = False

            model_name = f"""networks/AE/id=v{MODEL_VER:02}n{num:02}_in(64)_hid({min([layer.output.shape[1] for layer in model.layers])})_loss_MSE=
            (train={history.history['loss'][-1]:.5f},
            val={history.history['val_loss'][-1]:.5f},
            test={res_loss:.5f}).keras"""
            
            if history.history['loss'][-1] < min_train_loss and history.history['val_loss'][-1] < min_val_loss:
                min_train_loss = history.history['loss'][-1]
                min_val_loss = history.history['val_loss'][-1]
                cont = True
            if history.history['val_loss'][-1] < min_val_loss:
                min_val_loss = history.history['val_loss'][-1]
                cont = True
            if history.history['loss'][-1] < min_train_loss:
                min_train_loss = history.history['loss'][-1]
                cont = True
                
            print(f'\tEpochs: {len(history.history["loss"])}')
            print(f"\tloss_MSE=(train={history.history['loss'][-1]:.5f},val={history.history['val_loss'][-1]:.5f},test={res_loss:.5f})")
            
            
            if cont:
                model.save(model_name)
                num+=1
                cont=False
                #print(f'Start learning rate: {learning_rate}',"|"*5)
                #print(f'Batch size: {batch_size}')
                #print(f'Epochs: {len(history.history["loss"])}')
                #print(f"loss_MSE=(train={history.history['loss'][-1]:.5f},val={history.history['val_loss'][-1]:.5f},test={res_loss:.5f})")
                #print()
                continue
    print()'''

'''
Seed: 210 ||||||||||
Start learning rate: 0.005 |||||
Batch size: 16
Epochs: 24
loss_MSE=(train=0.00141,val=0.00098,test=0.00138)
'''

In [ ]:
'''name = 'id=v05n01_in(64)_hid(32)_loss_MSE=(train=0.00027,val=0.00023,test=0.00029)'
print(re.search(fr'hid\({ENCODED_SIZE}\)', name))
print([int(item) for item in re.findall(fr'id=v(\d+)n(\d+)', name)[0]])'''

In [ ]:
'''HP_SEED = hp.HParam('seed', hp.Discrete(np.arange(0,1000,10)))
HP_LEARNING_RATE = hp.HParam('learning_rate', hp.Discrete(0.0025, 0.005))
HP_BATCH_SIZE = hp.HParam('batch_size', hp.Discrete(np.arange(2,10,2)**2))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))

METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer(PATH_TO_SAVE_MODEL/'logs'/'hparam_tuning').as_default():
  hp.hparams_config(
    hparams=[HP_SEED, HP_LEARNING_RATE, HP_BATCH_SIZE, HP_OPTIMIZER],
    metrics=[hp.Metric('mean_squarred_error', display_name='MSE')],
  )''';

'''def train_test_model(hparams, dataset, callback_list):
  model = get_model(hparams[HP_LEARNING_RATE])
    
  model.compile(
      optimizer=hparams[HP_OPTIMIZER],
      loss='sparse_categorical_crossentropy',
      metrics=['accuracy'],
  )

    model.fit(dataset['train']['x'], dataset['train']['x'],
              batch_size = batch_size, 
              epochs = 60, 
              verbose = 0, 
              shuffle=True,
              validation_data = (dataset['val']['x'], dataset['val']['x']), 
              callbacks = callback_list)
    
  return model.evaluate(dataset['test']['x'], dataset['test']['x'])''';

'''def run(run_dir, hparams):
  with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hparams)  # record the values used in this trial
    accuracy = train_test_model(hparams)
    tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)''';

In [ ]:
'''FONT_SIZE = 15
fig, ax = plt.subplots()

fig.set_figwidth(12)
fig.set_figheight(8)

ax.plot(history.history['loss'], 
         label='Train dataset',  linewidth=1.5, color='blue')
ax.plot(history.history[f'val_loss'], linestyle = '--', 
     label='Validation dataset',  linewidth=3, color='red')
ax.set_xlabel('Epoch number', fontsize=FONT_SIZE)
ax.set_ylabel(f'Loss value', fontsize=FONT_SIZE)
ax.set_title(f"Learning process loss plot. Test dataset value = {res_loss:.4f} (MSE)", fontsize=FONT_SIZE, pad=15)

ax.patch.set_alpha(0)

#  Устанавливаем форматирование делений:
ax.tick_params(axis='both', which='both', labelsize = FONT_SIZE)

# Вывод и настройка сетки
ax.minorticks_on()
ax.grid(which='major', linewidth=2)
ax.grid(which='minor', color = 'gray', linestyle = ':')

ax.legend(fontsize = FONT_SIZE, facecolor = "white", loc = 'upper right')

plt.show()''';